<a href="https://colab.research.google.com/github/SeungHwaJung/Auto-Mobile-/blob/main/0718_python_yolo8%2C11%2C12_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# 📦 라이브러리 설치
!pip install -q ultralytics yt-dlp opencv-python pandas

# 📁 기본 임포트
import os
import cv2
import time
import pandas as pd
from ultralytics import YOLO
from IPython.display import Markdown, display

# ----------------------------------------------------------------------
# 🎥 1. 유튜브 영상 다운로드
# ----------------------------------------------------------------------
def download_youtube_video(youtube_url, output_path='/content'):
    import yt_dlp
    from datetime import datetime

    safe_filename = f"youtube_video_{datetime.now().strftime('%Y%m%d_%H%M%S')}.mp4"
    full_path = os.path.join(output_path, safe_filename)

    ydl_opts = {
        'outtmpl': full_path,
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
        'merge_output_format': 'mp4',
        'quiet': True,
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([youtube_url])
        print(f"✅ 다운로드 완료: {full_path}")
        return full_path
    except Exception as e:
        print(f"❌ 다운로드 실패: {e}")
        return None


# ----------------------------------------------------------------------
# 🧠 2. FPS 측정 함수
# ----------------------------------------------------------------------
def measure_fps(model, video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    start_time = time.time()

    processed_frames = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        model.predict(frame, verbose=False)
        processed_frames += 1

    cap.release()
    elapsed = time.time() - start_time
    fps = processed_frames / elapsed if elapsed > 0 else 0
    return fps, elapsed, processed_frames

# ----------------------------------------------------------------------
# 🚀 3. 전체 수행 함수
# ----------------------------------------------------------------------
def run_fps_comparison(youtube_url):
    # 1. 유튜브 영상 다운로드
    video_path = download_youtube_video(youtube_url)

    # 2. YOLO 모델 로드
    model_v8  = YOLO("yolov8n.pt")
    model_v11 = YOLO("yolo11n.pt")


    # 3. FPS 측정
    fps_v8,  time_v8,  frames_v8  = measure_fps(model_v8,  video_path)
    fps_v11, time_v11, frames_v11 = measure_fps(model_v11, video_path)


    # 4. 테이블 작성
    df = pd.DataFrame({
    "Model": ["YOLOv8n", "YOLOv11n"],
    "Processed Frames": [frames_v8, frames_v11],
    "Elapsed Time (s)": [time_v8, time_v11],
    "FPS": [fps_v8, fps_v11],
    "Δ FPS vs v8 (%)": [
        0,
        round((fps_v11 - fps_v8) / fps_v8 * 100, 2)
    ]
})

    # 5. 마크다운 저장
    md_content = df.to_markdown(index=False)
    with open("fps_comparison.md", "w") as f:
        f.write("# YOLO 모델 FPS 비교\n\n")
        f.write("※ 기준 모델: YOLOv8n\n\n")
        f.write(md_content)

    # 6. Colab에서 출력
    display(Markdown(f"# ✅ YOLO 모델 FPS 비교 결과\n\n{md_content}"))

# ----------------------------------------------------------------------
# ▶️ 실행: 유튜브 링크 입력 후 실행
# ----------------------------------------------------------------------
youtube_url = "https://www.youtube.com/watch?v=Zl-AgE-8oUg"
run_fps_comparison(youtube_url)

ERROR: [youtube] Zl-AgE-8oUg: Sign in to confirm you’re not a bot. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


❌ 다운로드 실패: ERROR: [youtube] Zl-AgE-8oUg: Sign in to confirm you’re not a bot. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


ZeroDivisionError: float division by zero